In [1]:
# 텐서플로우 관련
import tensorflow as tf
import tensorflow_hub as hub

# 이미지 다운로드
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

import pandas as pd

# F이미지에 그림 그리기
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# 시간 관련
import time

import pymongo

In [2]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [3]:
# 이미지 보여 주기

def display_image(image):
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)


# url 
def download_and_resize_image(url,display=False):
    _, filename = tempfile.mkstemp(suffix=".jpg")
    response = urlopen(url)
    image_data = response.read()
    image_data = BytesIO(image_data)
    pil_image = Image.open(image_data)
    pil_image = ImageOps.fit(pil_image, (pil_image.size[0], pil_image.size[1]), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert("RGB")
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    print("Image downloaded to %s." % filename)
    if display:
        display_image(pil_image)
    return filename




def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
    
    
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.

    
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
        text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())

    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
            draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
    return image

In [4]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img


def run_detector(detector, path):
    
    img = load_img(path)

    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    start_time = time.time()
    result = detector(converted_img)
    end_time = time.time()
    
    global result1

    result1 = {key:value.numpy() for key,value in result.items()}
    
    

    print("Found %d objects." % len(result["detection_scores"]))
    print("Inference time: ", end_time-start_time)
    print(converted_img.shape)

    image_with_boxes = draw_boxes(
    img.numpy(), result1["detection_boxes"],
    result1["detection_class_entities"], result1["detection_scores"])

In [5]:
# 인스타그램 사진 불러오고 저장

def insta_picture(image_url, save_path):
    downloaded_image_path = download_and_resize_image(image_url, False)
    
    run_detector(detector, downloaded_image_path)
    
    result_final = pd.DataFrame(result1["detection_boxes"])
    result_final.columns = ["ymin", "xmin", "ymax", "xmax"]
    img_color = Image.open(downloaded_image_path)
    result_final["xmin"] = round(result_final["xmin"]*img_color.size[0])
    result_final["xmax"] = round(result_final["xmax"]*img_color.size[0])
    result_final["ymin"] = round(result_final["ymin"]*img_color.size[1])
    result_final["ymax"] = round(result_final["ymax"]*img_color.size[1])
    result_final_array = result_final.values
    

    empty_df = pd.DataFrame(columns = ["ymin", "xmin", "ymax", "xmax", "score"])

    for i in range(min(result1["detection_boxes"].shape[0], 10)):
        if result1["detection_class_entities"][i] == b'Footwear':
            if result1["detection_scores"][i] > 0.1:
                ymin, xmin, ymax, xmax = tuple(result_final_array[i])
                empty_df.loc[i] = [ymin, xmin, ymax, xmax, result1["detection_scores"][i]]
            
    empty_df = empty_df.sort_values(by="score", ascending=False).reset_index(drop=True)
    
    empty_df = empty_df.astype({'ymin':'int'})
    empty_df = empty_df.astype({'xmin':'int'})
    empty_df = empty_df.astype({'ymax':'int'})
    empty_df = empty_df.astype({'xmax':'int'})
    
    try:
        xmin = empty_df.loc[0, "xmin"]
        ymin = empty_df.loc[0, "ymin"]
        xmax = empty_df.loc[0, "xmax"]
        ymax = empty_df.loc[0, "ymax"]
        
        croppedImage=img_color.crop((xmin, ymin, xmax, ymax))
        croppedImage.save(f"{save_path}.png", 'png')
        
    except (ValueError, KeyError):
        pass

(1) 앞의 함수 정의 후,

insta_picture("https://kream-phinf.pstatic.net/MjAyMjA3MTNfMjg3/MDAxNjU3NjkyNzg1MDkz.tvSfFyMTUKUUpWAGkEUWD2BKZVzupuKrYS5QqI8kf4gg._AHnLfYmYT02iE2le1ZrRyYBE-cbFuX-nnX1DJm8Ymgg.JPEG/p_423401e117a141a09b2ba332b9c20266.jpeg?type=l", save_path)로 사용

In [15]:
user = 'team04'
pw = '1111'
host = 'ec2-54-95-8-243.ap-northeast-1.compute.amazonaws.com'
client = pymongo.MongoClient(f'mongodb://{user}:{pw}@{host}:27017/')
db = client.resell

In [21]:
df = pd.DataFrame(db.insta.find({'new': 1}, {'_id': 0, 'img_url': 1}))
df

,img_url
0,https://scontent-nrt1-1.cdninstagram.com/v/t51...
1,https://scontent-nrt1-1.cdninstagram.com/v/t51...
2,https://scontent-nrt1-1.cdninstagram.com/v/t51...


In [31]:
list(df['img_url'])

['https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/295963742_1098282974113388_6318950424070481537_n.jpg?stp=dst-jpg_e15_s480x480&cb=2d435ae8-ef10543b&_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=fAhBqp9gdgYAX-rBGzX&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjg5MDc2OTY0MjE5ODcwMDk3Ng%3D%3D.2-ccb7-5&oh=00_AT_pVvqaC9IQlUDuTlSqhoQdjjCxKWFRsrQU_MYm5i-uiQ&oe=62E73897&_nc_sid=30a2ef',
 'https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/295773272_4709811405787607_6546210025068060674_n.jpg?stp=dst-jpg_e35&cb=2d435ae8-ef10543b&_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=2IsLZrxuGBMAX8J0SST&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=Mjg5MDc3MjQxMTg5MDU4NTAwOQ%3D%3D.2-ccb7-5&oh=00_AT8duQZLI5Vkiyy0tmQt-jmOvFxPDFIDO1cVwcjCZ8unKw&oe=62E6E161&_nc_sid=30a2ef',
 'https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/295707628_626856045449042_2322377478982896820_n.jpg?stp=dst-jpg_e35&cb=2d435ae8-ef10543b&_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=c9rDG5i

In [32]:
insta_picture(df.loc[2, 'img_url'], 'second')

1
Image downloaded to C:\Users\yhn03\AppData\Local\Temp\tmpcnxcmc0q.jpg.
Found 100 objects.
Inference time:  5.503957986831665
(1, 404, 320, 3)
Font not found, using default font.
2


In [23]:
for idx, row in df.iterrows():
    img_url = row['img_url']
    insta_picture(img_url, 'first')
    break

1
Image downloaded to C:\Users\yhn03\AppData\Local\Temp\tmp0ahoxlu6.jpg.
Found 100 objects.
Inference time:  5.18159031867981
(1, 480, 480, 3)
Font not found, using default font.
2


In [24]:
# 이미지 조정

def image_resize(save_path):
    im = Image.open(f'{save_path}.png')
    im = im.resize((100, 100))
    im.save(f'{save_path}.png')

(2) 앞의 함수 정의 후,
image_resize(save_path)로 사용

In [33]:
image_resize('second')

In [26]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

def model_predict(save_path):
    model = load_model('objection_detection_100_12.h5')
    image = cv2.imread(f"{save_path}.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    test_data = image / 255.0
    pred = model.predict(test_data[np.newaxis, ...])
    list = ["555088-105", "555088-134", "DM7866-140", "slipper"]
    print(list[np.argmax(pred)])

(3) 앞의 함수 정의 후,
model_predict(save_path)로 사용하면 모델 이름 추출

In [34]:
model_predict('second')

1/1 [==============================] - 0s 58ms/step
slipper


In [ ]:
db.insta.update()